# AIM

 COmponents of qutip various simulation (In all cases, no kerr correction to Generated pulses)
 
 1. Only ECD Hamiltonian
 1.5 Bare qubit-mode coupling
 2. Stark Shift
 3. Mode-Mode Coupling Terms (cross kerr)
 4.  transmon relaxation
 5. transmon dephasing


In [2]:
import sys 
sys.path.append('C:\\Users\\Eesh Gupta\\Documents\\RU Research\\Chakram')
sys.path.append('C:\\Users\\Eesh Gupta\\Documents\\RU Research\\Chakram\\ECD_control')

# For Prof. Chakram Desktop 
#sys.path.append('C:\\_Lib\\python\\ECD_control')
# sys.path.append('C:\\_Lib\\python')

In [3]:
%load_ext autoreload
%autoreload 2

In [18]:
import numpy as np
from qutip import *
#from DECD_paramV2 import FixedBatchOptimizer as BatchOptimizer
import matplotlib.pyplot as plt
from IPython.display import clear_output
import ast


In [7]:
#Modes Truncation
N1 = 6#15
N2 = 6#15

### Batch Optimizer

In [8]:
#get the params 
import pandas as pd
param_df = pd.read_csv('beam_splitters')

In [9]:
param_df

,task,layer,pulse_time,BO_fid,filenum
0,1,4,5,0.026118,0
1,1,6,5,0.005521,1
2,1,8,5,0.005272,2
3,2,4,5,0.012684,3
4,2,6,5,0.043766,4
5,2,8,5,0.008298,5
6,3,4,5,0.014820,6
7,3,6,5,0.012616,7
8,3,8,5,0.009694,8
9,4,4,5,0.024376,9


In [26]:
param_df_n = param_df[param_df['task'] == 1]
param_df_n_max = param_df_n[param_df_n['BO_fid'] ==min(param_df_n['BO_fid'])]
param_df_n_max.reset_index()
#param_df_n_max['BO_fid'] 


,index,task,layer,pulse_time,BO_fid,filenum
0,2,1,8,5,0.005272,2


### Get the Pulses

In [10]:
from Simulation_Classes_Two_ModeV4 import *

### Calculator (Not Needed)
Since Batch Optimizer agrees generally with calc

In [8]:
from qutip import *

In [9]:
def mod_disp_op(disp, n_q, n_c):
    '''
    Returns displacement operator using baken campbell formula
    '''
    pauli_like_x = (create(n_c) + destroy(n_c))
    pauli_like_y = (1j)*(create(n_c) - destroy(n_c))
    comm = (1/2)*((pauli_like_x*pauli_like_y) - (pauli_like_y*pauli_like_x))
    re = np.real(disp)
    im = np.imag(disp)

    first = (1j*im*pauli_like_x).expm()
    second = (-1j*re*pauli_like_y).expm()
    third = ((im*re)*(-1)*comm).expm()
    return first*second*third

def disp_op(disp, n_q, n_c):
    '''
    Returns displacement operator for specified displacement
    '''
    #disp = normalize_complex(disp)
    exponent = (disp*create(n_c))- (np.conjugate(disp)*destroy(n_c))
    return  exponent.expm()

def cond_disp_op(beta, n_q, n_c1, n_c2,mode, use_mod):
    '''
    Returns cond displacement operator for specified real displacement
    '''
    disp = beta/2
    
    n_c = n_c1 if mode == 1 else n_c2
    
    if use_mod: 
        d = mod_disp_op(disp= disp, n_q = n_q, n_c = n_c) #Baker Campbell Approx
    else: 
        d = disp_op(disp= disp, n_q = n_q, n_c = n_c)
    d_adjoint = d.dag()
    
    if mode == 1: 
        left = tensor(create(n_q), d, qeye(n_c2)) #D(beta/2)|e><g|
        right = tensor(destroy(n_q), d_adjoint, qeye(n_c2)) #D(-beta/2)|g><e|   ...not sure d_adjoint(alpha) = d(-alpha) if use Baker
    
    else: #mode ==2 
        left = tensor(create(n_q), qeye(n_c2), d)
        right = tensor(destroy(n_q),  qeye(n_c2), d_adjoint)
    return left+right

def qubit_rot(phi, theta, n_q, n_c1, n_c2):
    '''
    Returns qubit rotation
    '''
    rot = (np.cos(phi)*sigmax()) + (np.sin(phi)*sigmay())
    exp = (-1.0j)*(theta/2)*(rot)
    return tensor(exp.expm(), qeye(n_c1), qeye(n_c2) )

def normalize_complex(number):
    '''
    Returns radius r of complex number z = r*e^iphi
    '''
    return np.sqrt(number.real**2 + (number .imag**2))

def dot(state1, state2):
    '''
    dotting both states
    '''
    fid = state1.overlap(state2)
    return fid*np.conjugate(fid)
def evolve(initial_state, betas, gammas, phis, thetas, n_q, n_c1, n_c2, use_mod = False):
    '''
    Operates on initial_state with ECD(beta_n)*R(phi_n, theta_n) *...........*ECD(beta_1)*R(phi_1, theta_1)
    '''
    n = len(betas)
    state = initial_state
    for i in range(n):
        beta = betas[i]
        gamma = gammas[i]
        phi = phis[i]
        theta = thetas[i]
        
  
    
        #state = cond_disp_op(gamma, n_q, n_c1,n_c2,mode = 2,use_mod = use_mod)*cond_disp_op(beta, n_q, n_c1,n_c2, mode = 1,use_mod = use_mod)*qubit_rot(phi, theta, n_q, n_c1, n_c2)*state
        state = cond_disp_op(beta, n_q, n_c1,n_c2, mode = 1,use_mod = use_mod)*cond_disp_op(gamma, n_q, n_c1,n_c2,mode = 2,use_mod = use_mod)*qubit_rot(phi, theta, n_q, n_c1, n_c2)*state
           
    return state
        
        

In [10]:
# initial = tensor(basis(2, 0), basis(N1, 0), basis(N2, 0))
# target = tensor(basis(2, 0), basis(N1, 1), basis(N2, 0))
# pi_target = tensor(basis(2, 1), basis(N1, 1), basis(N2, 0))

In [11]:
def Calculator(betas,gammas, phis, thetas): 
    '''
    Returns fidelity using qutip matrix algebra
    '''
    rho_calc = evolve(tensor(basis(2,0), initial),betas,gammas, phis, thetas,n_q = 2,n_c1 = N1, n_c2 = N2,
                   use_mod = False)
    return dot(tensor(basis(2,0),target), rho_calc)

In [12]:
#Calculator(betas,gammas, phis, thetas)

# Main
Vary conditions, see differences in fidelities

In [11]:
def two_mode_state(fock1, fock2, qubit_g = True): 
    '''
    Returns g x fock1 x fock 2
    
    #takes N1, N2 from global
    '''
    psi_1 = basis(N1,fock1) #initial state
    psi_2 = basis(N2,fock2)
    return tensor(basis(2,0), psi_1, psi_2)

In [12]:
errors = ['mode coupling',
          'transmon relax',
          'transmon dephasing',
          'cavities relax',
          'cavities dephasing'
          ]  

In [37]:
columns = ['task', 'bo_fidelity','only ecd'] +errors
df = pd.DataFrame([[None for i in range(len(columns))]], columns=columns)

In [14]:
df

,bo_fidelity,only ecd,bare qubit mode coupling,stark shift,mode coupling,transmon relax,transmon dephasing,cavities relax,cavities dephasing
0,None,None,None,None,None,None,None,None,None


4

In [29]:
fname = "Noise_sim_one_by_one"
df.to_csv(fname, index=False)
angles_filename_prefix = 'Data/angles'

In [38]:

N = max(param_df['task'])#how many beam splitters
for n in range(1, N+1): 
   
    #Choose angles which give max fid for n0->0n task
    param_df_n = param_df[param_df['task'] == n]
    param_df_n_max = param_df_n[param_df_n['BO_fid'] ==max(param_df_n['BO_fid'])]
    param_df_n_max = param_df_n_max.reset_index()
    filenum= param_df_n_max['filenum'][0] 
    bo_fid =  param_df_n_max['BO_fid'][0]   
    #Then get pulses
    
    T1_mode1 = 2e+6 #2ms
    T1_mode2 = 2e+6 #2ms
    
    angles_filename = angles_filename_prefix + '_' + str(filenum) + '.txt'

    pulse_sim = ecd_pulse_two_mode(param_file = angles_filename,
                                  kappa1 = 1/T1_mode1,
                                  kappa2 = 1/T1_mode2)
    pulse_sim.get_pulses()
    initial = two_mode_state(n, 0)
    target = two_mode_state(0, n)
    
      
    #Only ECD term
    print('Only ECD term in Qutip')
    qutip_sim = qutip_sim_two_mode(n_q = 2, n_c1 = N1, n_c2 = N2,
                                   alpha1 = pulse_sim.alpha1,
                                   alpha2 = pulse_sim.alpha2,
                                   qubit_pulse = pulse_sim.qubit_dac_pulse_GHz,
                                   save_states = False)
    qutip_sim.me_solve(initial = initial)
    ecd_fid = np.real(qutip_sim.get_fidelity(target))
    print(ecd_fid)
    
    
#     print('Now adding qubit-mode bare coupling')
#     qutip_sim = qutip_sim_two_mode(n_q = 2, n_c1 = N1, n_c2 = N2,
#                                    alpha1 = pulse_sim.alpha1,
#                                    alpha2 = pulse_sim.alpha2,
#                                    qubit_pulse = pulse_sim.qubit_dac_pulse_GHz,
#                                    save_states = False)
#     qutip_sim.add_bare_qubit_mode_coupling()
#     qutip_sim.me_solve(initial = initial)
#     qm_fid = np.real(qutip_sim.get_fidelity(target))
#     print(qm_fid)
    
#     print('Now adding Stark Shifts')
#     qutip_sim = qutip_sim_two_mode(n_q = 2, n_c1 = N1, n_c2 = N2,
#                                    alpha1 = pulse_sim.alpha1,
#                                    alpha2 = pulse_sim.alpha2,
#                                    qubit_pulse = pulse_sim.qubit_dac_pulse_GHz,
#                                    save_states = False)
#     qutip_sim.add_stark_shift()
#     qutip_sim.me_solve(initial = initial)
#     ss_fid = np.real(qutip_sim.get_fidelity(target))
#     print(ss_fid)
    
    print('Now adding Mode Mode Coupling')
    qutip_sim = qutip_sim_two_mode(n_q = 2, n_c1 = N1, n_c2 = N2,
                                   alpha1 = pulse_sim.alpha1,
                                   alpha2 = pulse_sim.alpha2,
                                   qubit_pulse = pulse_sim.qubit_dac_pulse_GHz,
                                   save_states = False)
    qutip_sim.add_mode_mode_coupling()
    qutip_sim.me_solve(initial = initial)
    mm_fid = np.real(qutip_sim.get_fidelity(target))
    print(mm_fid)
    
    
    print('Now adding Transmon Relaxation')
    qutip_sim = qutip_sim_two_mode(n_q = 2, n_c1 = N1, n_c2 = N2,
                                   alpha1 = pulse_sim.alpha1,
                                   alpha2 = pulse_sim.alpha2,
                                   qubit_pulse = pulse_sim.qubit_dac_pulse_GHz,
                                   save_states = False)
    qutip_sim.add_qubit_relaxation(T1 = 85e+3)
    qutip_sim.me_solve(initial = initial)
    tr_fid = qutip_sim.get_fidelity(target)
    print(tr_fid)
    
    print('Now adding Transmon Dephasing')
    qutip_sim = qutip_sim_two_mode(n_q = 2, n_c1 = N1, n_c2 = N2,
                                   alpha1 = pulse_sim.alpha1,
                                   alpha2 = pulse_sim.alpha2,
                                   qubit_pulse = pulse_sim.qubit_dac_pulse_GHz,
                                   save_states = False)
    qutip_sim.add_qubit_dephasing(T1 = 98e+3, Techo = 120e+3)
    qutip_sim.me_solve(initial = initial)
    tp_fid = qutip_sim.get_fidelity(target)
    print(tp_fid)
    
    print('Now Adding Cavity Relaxation')
    qutip_sim = qutip_sim_two_mode(n_q = 2, n_c1 = N1, n_c2 = N2,
                                   alpha1 = pulse_sim.alpha1,
                                   alpha2 = pulse_sim.alpha2,
                                   qubit_pulse = pulse_sim.qubit_dac_pulse_GHz,
                                   save_states = False)
    qutip_sim.add_cavity_relaxation(T1_mode1 = 2e+6, T1_mode2 = 2e+6)
    qutip_sim.me_solve(initial = initial)
    cr_fid = qutip_sim.get_fidelity(target)
    print(cr_fid)
    
    print('Now Adding Cavity Dephasing')
    qutip_sim = qutip_sim_two_mode(n_q = 2, n_c1 = N1, n_c2 = N2,
                                   alpha1 = pulse_sim.alpha1,
                                   alpha2 = pulse_sim.alpha2,
                                   qubit_pulse = pulse_sim.qubit_dac_pulse_GHz,
                                   save_states = False)
    qutip_sim.add_cavity_dephasing(T1_mode1 = 2e+6, Techo_mode1 = 3e+6, T1_mode2 = 2e+6, Techo_mode2 = 3e+6) 
    qutip_sim.me_solve(initial = initial)
    cp_fid = qutip_sim.get_fidelity(target)
    print(cp_fid)
    
    
    
    #Saving data
    task = n
    new_row = [[task, bo_fid, ecd_fid, qm_fid, ss_fid, mm_fid, tr_fid, tp_fid, cr_fid, cp_fid]]
    df_new = pd.DataFrame(new_row, columns=columns)
    df = df.append(df_new, ignore_index = True)
    df.to_csv(fname, index=False)
    
    clear_output(wait = True)
    print(new_row)
    



[[4, 0.0364710912108421, 0.01419371325552242, 0.014851343766502706, 0.01623367766981226, 0.014193707928996371, 0.000199556842810381, 0.00020080873718758256, 0.00020135278996074037, 0.00020140361279846687]]


# DUMP

In [33]:
n = 1
#Choose angles which give max fid for n0->0n task
param_df_n = param_df[param_df['task'] == n]
param_df_n_max = param_df_n[param_df_n['BO_fid'] ==min(param_df_n['BO_fid'])]
param_df_n_max = param_df_n_max.reset_index()
filenum= param_df_n_max['filenum'][0] 
bo_fid =  param_df_n_max['BO_fid'][0]   
#Then get pulses

T1_mode1 = 2e+6 #2ms
T1_mode2 = 2e+6 #2ms

angles_filename = angles_filename_prefix + '_' + str(filenum) + '.txt'

pulse_sim = ecd_pulse_two_mode(param_file = angles_filename,
                              kappa1 = 1/T1_mode1,
                              kappa2 = 1/T1_mode2)
#pulse_sim.get_pulses()
initial = two_mode_state(n, 0)
target = two_mode_state(0, n)

In [35]:
filenum

2

In [34]:
pulse_sim.betas

array([-1.69516397-0.71398467j,  0.55999124+0.06270218j,
       -0.92010558-0.12129299j,  1.50502276+1.68139863j,
       -1.50179398+1.9599154j , -2.42324042-1.3498385j ,
       -0.82987183-0.42289576j,  1.46341646-0.04244507j])

In [21]:
lf = pd.read_csv(fname)
result_df = lf.drop(0)

In [22]:
result_df

,exact fidelity,bo_fidelity,betas,gammas,phis,thetas,only ecd,bare qubit mode coupling,stark shift,mode coupling,transmon relax,transmon dephasing,cavities relax,cavities dephasing
1,0.995586,0.995598,[-0.00262321-0.00144306j 0.02715145-0.0185476...,[-0.22359522-4.9973166e-01j -0.7231863 +1.4162...,[ 0. 1.3961155 1.2453349 1.0953438 -...,[-1.5561874 2.275472 1.7466791 2.030403...,(0.9823420776469016+0j),(0.9832311532339598+0j),(0.9539217486165369+0j),(0.9823420800332973+0j),0.900607,0.959755,0.964751,0.966590
2,0.996460,0.996470,[-6.4737914e-04-3.2210814e-05j -5.2747920e-01+...,[ 5.0107450e-01+1.6034484e-02j -1.5052351e+00-...,[ 0. -1.5745307 -0.7655482 1.881175...,[ 1.5732338 -1.2030032 -3.1294155 3.1376164 -...,(0.9872561739264472+0j),(0.9888673507517175+0j),(0.9758650199326269+0j),(0.9872561691033227+0j),0.898399,0.968749,0.974437,0.977177
3,0.994936,0.995629,[ 1.0140339e-03+5.3937244e-04j -4.4833508e-04-...,[ 2.636092 +0.4190833j 0.01244382-0.432337j...,[ 0. -0.7320552 1.9630716 -2.370027 ...,[-0.0271492 1.5207536 -1.6562961 2.018657 -...,(0.9860482692230856+0j),(0.9868663279406935+0j),(0.9760869449864844+0j),(0.986048324988694+0j),0.890441,0.966025,0.970544,0.973640
4,0.995886,0.995833,[ 6.5347767e-06-1.30088590e-04j 2.0591020e-03...,[-3.8990060e-01-3.9146677e-01j 5.7316244e-01-...,[ 0. -1.5533143 -1.5631359 -1.597816...,[ 1.5757606e+00 -1.1407130e+00 1.5744059e+00 ...,(0.9830461975480413+0j),(0.9786824203190096+0j),(0.9716241824785214+0j),(0.9830461807947878+0j),0.881018,0.959376,0.962301,0.968325
5,0.996191,0.996193,[ 6.5717466e-02+1.5812736j -1.1736141e-02+0.0...,[ 1.418391 +0.24241747j 0.33849597-0.9158984...,[ 0. 0.81113863 -2.1159046 -1.300121...,[ 3.136113 -1.5799026 0.99619174 0.772885...,(0.9951864811660737+0j),(0.993798660029528+0j),(0.9951332086068738+0j),(0.9951864809402454+0j),0.906618,0.984200,0.988287,0.991219
6,0.996768,0.996779,[ 1.38473530e-03+1.6311259e-03j 1.09282150e+0...,[ 9.3426320e-02-4.8979080e-01j 3.3189622e-01+...,[ 0. 1.5340307 -2.4499028 -0.181198...,[ 1.5795605 -1.1847471 3.134495 1.818692...,(0.9895283416874554+0j),(0.9859483712842132+0j),(0.982407793178679+0j),(0.9895283274706616+0j),0.892199,0.971686,0.977033,0.980494
7,0.996423,0.996423,[-0.02452667-2.7897577e-03j -0.00146851+1.7478...,[-3.7821588e-01-2.5588945e-01j -6.1016450e-01+...,[ 0. 1.5692036 1.5720084 -1.343168...,[-1.5716397 1.3233197 -2.0805616 0.003912...,(0.9921654483498581+0j),(0.9927585803823321+0j),(0.9803020223624959+0j),(0.9921654432847098+0j),0.908355,0.977804,0.984134,0.985696
8,0.996415,0.996425,[ 1.4479039e-03-4.1262887e-04j 3.3884450e-04-...,[-0.4419903 +0.3685562j -0.5735252 -0.605279j...,[ 0. 1.5909874 1.6046488 1.625607...,[-1.5605228 -2.0086124 -1.5044947 0.602578...,(0.9952955708757404+0j),(0.9954810016731153+0j),(0.9952296579328929+0j),(0.9952955710493228+0j),0.910534,0.984494,0.989321,0.993554
9,0.996539,0.996526,[ 0.01475754-0.04110372j 0.4100612 -0.2312351...,[-0.43818113+0.2351073j -2.1519425 +1.3530631...,[ 0. -1.567324 1.4665563 1.599829...,[-1.5563223 -1.123174 -0.00695944 -3.125749...,(0.9770431426868857+0j),(0.9836069118366287+0j),(0.9733678234417471+0j),(0.9770431381106969+0j),0.883177,0.949441,0.953105,0.958011
10,0.995410,0.995419,[-0.13434817-1.1718981j 0.06487343-0.0059511...,[ 0.3952284 +1.0193292j 0.39807162+0.1077477...,[ 0. -0.25606656 1.6486723 1.125031...,[-0.02907538 1.589442 1.7153823 1.757087...,(0.9747704833695442+0j),(0.977191267816673+0j),(0.9680572353989496+0j),(0.9747704610147359+0j),0.885569,0.944926,0.948887,0.951441


In [23]:
fname = "Noise_sim_one_by_one__15_level_sep_15.txt"
result_df.to_csv(fname, index=False)